In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import os

In [6]:

metadata = pd.read_csv('metadata.csv')
image_dir = 'images/imgs_part_1/imgs_part_1/'
image_files = set(os.listdir(image_dir))

def image_exists(img_id):
    return img_id in image_files

metadata['image_exists'] = metadata['img_id'].apply(image_exists)
metadata_filtered = metadata[metadata['image_exists']]
metadata_filtered = metadata_filtered.drop(columns=['image_exists'])
metadata_filtered.to_csv('filtered_metadata.csv', index=False)
metadata = pd.read_csv('filtered_metadata.csv')

# Filter relevant columns and map diagnostic to binary cancer labels
data = metadata[[
                #'background_father', 'background_mother','gender',  'biopsed' 
                 'age', 'img_id', 'diagnostic'
                 ,'smoke', 'drink', 'skin_cancer_history', 'cancer_history', 'has_piped_water', 'has_sewage_system'
                 , 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation'
                 ]]
data['diagnostic'] = data['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC':1, 'ACK': 0, 'NEV': 0, 'SEK': 0})
                                               #1,      0.8       0.6       0.4       0.2        0
data = data.dropna(subset=['diagnostic'])

data['image_path'] = image_dir + data['img_id']
#data['background_father'] = data['background_father'].astype('category').cat.codes
#data['background_mother'] = data['background_mother'].astype('category').cat.codes
#data['biopsed'] = data['biopsed'].astype('category').cat.codes
#data['gender'] = data['gender'].astype('category').cat.codes
data['smoke'] = data['smoke'].astype('category').cat.codes
data['drink'] = data['drink'].astype('category').cat.codes
data['skin_cancer_history'] = data['skin_cancer_history'].astype('category').cat.codes
data['cancer_history'] = data['cancer_history'].astype('category').cat.codes
data['has_piped_water'] = data['has_piped_water'].astype('category').cat.codes
data['has_sewage_system'] = data['has_sewage_system'].astype('category').cat.codes
data['region'] = data['region'].astype('category').cat.codes
data['itch'] = data['itch'].astype('category').cat.codes
data['grew'] = data['grew'].astype('category').cat.codes
data['hurt'] = data['hurt'].astype('category').cat.codes
data['changed'] = data['changed'].astype('category').cat.codes
data['bleed'] = data['bleed'].astype('category').cat.codes
data['elevation'] = data['elevation'].astype('category').cat.codes


# Separate features and target
X_tabular = data[['age'
                #'background_father', 'background_mother',  'biopsed'
                  ,'smoke', 'drink', 'skin_cancer_history', 'cancer_history', 'has_piped_water', 'has_sewage_system'
                  , 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation'
                  ]].values
y = data['diagnostic'].values
image_paths = data['image_path'].values

# Split data into training and validation sets
X_train_tabular, X_val_tabular, y_train, y_val, train_image_paths, val_image_paths = train_test_split(
    X_tabular, y, image_paths, test_size=0.4, random_state=42)

C:\Users\umutg\AppData\Local\Temp\ipykernel_22184\479934303.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['diagnostic'] = data['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC':1, 'ACK': 0, 'NEV': 0, 'SEK': 0})


In [7]:
class CustomDataset(Dataset):
    def __init__(self, tabular_data, image_paths, labels, transform=None):
        self.tabular_data = tabular_data
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        tabular_data = torch.tensor(self.tabular_data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        return tabular_data, image, label


In [8]:
# Define transforms for the image data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
    ,transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = CustomDataset(X_train_tabular, train_image_paths, y_train, transform=transform)
val_dataset = CustomDataset(X_val_tabular, val_image_paths, y_val, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [9]:
class MultimodalNetwork(nn.Module):
    def __init__(self):
        super(MultimodalNetwork, self).__init__()
        
        # ResNet50 model for image data
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final layer
        
        # Multi Layer Perceptron for tabular data
        self.fc_tabular = nn.Sequential(
            nn.Linear(14, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU()
        )
        
        # Combined layers
        self.fc_combined = nn.Sequential(
            nn.Linear(2048 + 32, 128), 
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
    def forward(self, tabular_data, images):
        img_features = self.resnet(images)
        tabular_features = self.fc_tabular(tabular_data)
        combined_features = torch.cat((img_features, tabular_features), dim=1)
        output = self.fc_combined(combined_features)
        return output


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = MultimodalNetwork().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for tabular_data, images, labels in train_loader:
        tabular_data, images, labels = tabular_data.to(device), images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(tabular_data, images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * tabular_data.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


c:\Users\umutg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\umutg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cpu
Epoch 1/10, Loss: 0.5844
Epoch 2/10, Loss: 0.5602


In [ ]:
model.eval()
val_loss = 0.0
correct = 0
total = 0

all_labels = []
all_predictions = []

with torch.no_grad():
    for tabular_data, images, labels in val_loader:
        tabular_data, images, labels = tabular_data.to(device), images.to(device), labels.to(device)
        
        outputs = model(tabular_data, images).squeeze()
        loss = criterion(outputs, labels)
        
        val_loss += loss.item() * tabular_data.size(0)
        predicted = (outputs > 0.5).float()
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_loss = val_loss / len(val_loader.dataset)
accuracy = correct / total
print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')

# Print the confusion matrix
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
print(conf_matrix)


Validation Loss: 0.3122, Accuracy: 0.8587
[[410  62]
 [ 68 380]]


In [ ]:
print(f'Total training samples: {len(train_dataset)}')
print(f'Total validation samples: {len(val_dataset)}')
print(f'Total training batches: {len(train_loader)}')
print(f'Total validation batches: {len(val_loader)}')
labels
predicted

Total training samples: 1378
Total validation samples: 920
Total training batches: 44
Total validation batches: 29


tensor([1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
        1., 0., 1., 1., 0., 0.])